In [1]:
from smartcard import ATR
from smartcard.System import readers
from smartcard.CardConnection import CardConnection

In [3]:
rarr=readers()
if len(rarr) == 0:
    print("No readers detected")
else:
    reader = rarr[0]
    print(reader)

Generic Smart Card Reader Interface


In [11]:
protocol = CardConnection.T1_protocol
connection = reader.createConnection()
connection.connect(protocol)

In [12]:
atr=bytes(connection.getATR())
atr.hex()

'3bfe1800008031fe45803180664090a5102e1083019000f2'

In [13]:
def select(appletID):
    data = list(bytes.fromhex(appletID))
    # Select:
    # CLA = 0x00
    # INS = 0xA4
    # P1 = 0x04
    # P2 = 0x00
    # Data = the instance AID
    cmd = [0x00, # CLA
              0xA4, # INS
              0x04, # P1
              0x00, # P2
              len(data), # Lc (content length)
          ] + data + [0x00]
    print(bytes(cmd).hex().upper())
    data, *sw = connection.transmit(cmd,protocol)
    data = bytes(data)
    sw = bytes(sw)
    if sw == b"\x90\x00":
        return data
    else:
        raise RuntimeError("Card responded with code %s and data \"%s\"" % (sw.hex(), data.hex()))

In [14]:
select("B00B5111CA01")

00A4040006B00B5111CA0100


b''

In [15]:
len(bytes.fromhex("B00B5111CA01"))

6

In [15]:
def request(APDU):
    cmd = list(bytes.fromhex(APDU))
    data, *sw = connection.transmit(cmd,protocol)
    data = bytes(data)
    sw = bytes(sw)
    if sw == b"\x90\x00":
        return data
    else:
        raise RuntimeError("Card responded with code %s and data \"%s\"" % (sw.hex(), data.hex()))

In [16]:
# get secret
request("B0A100000000")

b'shiny teapot taking care of your secrets'

In [14]:
request("00C0000028")

b'shiny teapot taking care of your secrets'

In [23]:
secret = "fkjshkjdshkfjshdkjfhsdjkfhkjsdhfjksdhfkjsdhfksdhfkjsdhfjsdshiny teapot taking care of your secrets"
print(len(secret))
data = bytes([len(secret)])+secret.encode()
data.hex()

98


'62666b6a73686b6a6473686b666a7368646b6a666873646a6b66686b6a736468666a6b736468666b6a736468666b736468666b6a736468666a73647368696e7920746561706f742074616b696e672063617265206f6620796f75722073656372657473'

In [24]:
len(b"shiny teapot taking care of your secrets")

40

In [25]:
# put secret
request("B0A20000"+data.hex()+"00")

RuntimeError: Card responded with code 6700 and data ""

In [ ]:
# get secret again
request("B0A100000000")

In [10]:
connection.disconnect()

In [ ]:
import smartcard

In [ ]:
smartcard.__file__

In [ ]:
connection.getProtocol()

In [16]:
bytes.fromhex("00c07368696e7920746561706f742074616b696e672063617265206f6620796f757220736563726574739000")

b'\x00\xc0shiny teapot taking care of your secrets\x90\x00'